In [15]:
import gurobipy as gp
from gurobipy import GRB
import math

In [2]:
def calculate_optimal_gurobi(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=100):
    m = gp.Model('box_constraint_test')
    m.setParam( 'OutputFlag', False )
    # Variables
    Bi = m.addVar(lb=lb, ub=ub, vtype=GRB.CONTINUOUS, name="Bi")
    Bi_supp = m.addVar(vtype=GRB.BINARY, name = '1[Bi]')
    Bi_sign = m.addVar(vtype=GRB.BINARY)
    Bi_pos = m.addVar(lb=0, ub=ub, vtype=GRB.CONTINUOUS)
    Bi_neg = m.addVar(lb=0, ub=-lb, vtype=GRB.CONTINUOUS)
    
    #Constraints
    m.addConstr(Bi_pos <= M*Bi_sign)
    m.addConstr(Bi_neg <= M*(1-Bi_sign))
    m.addConstr(Bi == Bi_pos - Bi_neg)

    m.addConstr(Bi >= -M*Bi_supp)
    m.addConstr(Bi <= M*Bi_supp)
    
    #Cost
    m.setObjective((B - Bi)*(B - Bi) + l0*Bi_supp + l1*(Bi_pos + Bi_neg) + l2*Bi*Bi, GRB.MINIMIZE)
    
    m.optimize()
    
    for v in m.getVars():
        if v.varName == "Bi":
            return v.x

In [140]:
def box(x, lb, ub):
    if x > ub:
        return ub
    elif x <= lb:
        return lb
    else:
        return x
    
def sign(x):
    if x < 0:
        return -1
    return 1

def calculate_closed_from(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    x = box(sign(Bi)*(abs(Bi) - l1)/(1 + 2*l2), lb, ub)
    
    #print(x)
    
    condition1_for_zero_Bi = (abs(Bi) - l1)/(1 + 2*l2) < math.sqrt(2*l0/(1+2*l2))
    
    #print(condition1_for_zero_Bi)
    if condition1_for_zero_Bi:
        return 0
    
    # Uneccessary step, just maintained for similairy to derivation
    condition2a_for_zero_Bi = not condition1_for_zero_Bi 
    
    delta = (abs(Bi) - l1)**2 - 2*l0*(1 + 2*l2)
    
     
    x_range_low = ((abs(Bi) - l1)*sign(Bi) - math.sqrt(delta))/(1 + 2*l2) 
    x_range_high = (((abs(Bi) - l1)*sign(Bi) + math.sqrt(delta))/(1 + 2*l2))
    
    #print(x_range_low, x_range_high)
    condition2b_for_zero_Bi = x_range_low <= x <= x_range_high
    
    if (condition2a_for_zero_Bi and condition2b_for_zero_Bi):
        return x
    else:
        return 0

In [141]:
def cost(B, Bi, l0, l1, l2):
    return (B - Bi)**2 + l0*(Bi!=0) + l1*abs(Bi) + l2*Bi**2

In [157]:
l0 = .452
l1 = .2231
l2 = 0.1231
B = 3
lb = -0.5
ub = 0.5

In [158]:
Bi_optimal = calculate_optimal(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_optimal, cost(B, Bi_optimal, l0, l1, l2))

0.5 6.844325


In [159]:
Bi_box = calculate_closed_from(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(B, Bi_box, l0, l1, l2))

0.5 6.844325


Test the implementations with 'hypothesis'. <br>
hypothesis will search through the space defined by @given(...) and try to find examples that fail. It will then search the "neighbors" of that example trying to make as many of the variables 0 as possible.

For example:
```python
def add(x, y):
    return x - y

@given(x=floats(0, 10), y=floats(0, 10))
def test_add(x, y):
    # x and y will be assigned floating values between 0, 10
    assert add(x, y) == x + y
    
test_add() # Run here
```

test_add will try different values of x and y until it finds a failure. It will then try to push x and y to zero to find the "smallest" error. It will successfully push 'x' to zero as the 'truthyness" of x-y == x+y does not depend on x. It only depends on y. Therefore, the failure it will find should be x=0 and y being very small.

In [198]:
def add(x, y):
    return x - y

@given(x=floats(0, 10), y=floats(0, 10))
def test_add(x, y):
    # x and y will be assigned floating values between 0, 10
    assert add(x, y) == x + y
    
test_add()

Falsifying example: test_add(
    x=0.0, y=2.2204460492503135e-15,
)


AssertionError: 

# Lets run on our problem

In [179]:
import hypothesis
from hypothesis import given, settings
from hypothesis.strategies import floats
import numpy as np

# Compare values

In [172]:
@given(l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(0, 10), lb=floats(-1, 0), ub=floats(0, 1))
def compare_B(l0, l1, l2, B, lb, ub):
    np.testing.assert_approx_equal(calculate_optimal(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub),
                                   calculate_closed_from(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub))

In [173]:
compare()

Falsifying example: compare(
    l0=0.011443775404216263,
    l1=0.0,
    l2=0.0,
    B=0.262548605167976,
    lb=0.0,
    ub=0.047969437336572454,
)


AssertionError: 
Items are not equal to 7 significant digits:
 ACTUAL: 0.047969437336572454
 DESIRED: 0.0

In [169]:
l0=0.2974940618184397
l1=0.0
l2=0.0
B=2.252696840873985
lb=0.0
ub=0.06702783687498594

0 5.074643056883632


In [170]:
Bi_box = calculate_closed_from(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(B, Bi_box, l0, l1, l2))

0 5.074643056883632


In [171]:
Bi_optimal = calculate_optimal(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_optimal, cost(B, Bi_optimal, l0, l1, l2))

0.06702783687498594 5.074643056860415


# Compare Objectives

In [193]:
@given(l0=floats(0, 10), l1 = floats(0, 10), l2 = floats(0, 10),
       B=floats(0, 10), lb=floats(-1, 0), ub=floats(0, 1))
@settings(max_examples=1000, derandomize=True)
def compare_obj(l0, l1, l2, B, lb, ub):
    global decimal
    Bi_optimal = calculate_optimal(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    Bi_box = calculate_closed_from(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
    
    np.testing.assert_almost_equal(cost(B, Bi_optimal, l0, l1, l2),
                                   cost(B, Bi_box, l0, l1, l2), decimal=decimal)

In [194]:
decimal = 2
compare_obj()

Falsifying example: compare_obj(
    l0=0.0,
    l1=0.5270922086437181,
    l2=0.0,
    B=0.5032392635277817,
    lb=0.0,
    ub=0.03366015095731623,
)


AssertionError: 
Arrays are not almost equal to 2 decimals
 ACTUAL: 0.23824654627383865
 DESIRED: 0.2532497563559841